In [1]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time
import datetime
from datetime import date

In [2]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [3]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]

def pad(data):
  length = 16 - (len(data) % 16)
  return data + chr(length).encode('utf-8')*length

def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

def encodeMessage(msg):
  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Erster Schritt: AES256 verschlüsseln
  iv = Random.new().read(AES.block_size)
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
  msg = msg.encode('utf-8')
  crypt = iv + cipher.encrypt(pad(msg))

  # Zweiter Schritt: base64 enkodieren
  msgb64 = base64.b64encode(crypt).decode('utf-8')
  return msgb64

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln
    msg = decodeMessage(contents.read())
  
    #JSON-String umwandeln in dict
    folder_dict = json.loads(msg)
    
    # Verbindung trennen
    connection.close()

    return folder_dict

#Funktion zur erstellung des SQL-Statements
def sql_data(contents, cur):
  #content wird Zeile für Zeile verarbeitet
  for line in contents.read().split(b'\n'):
      if line != b'':
          #Message wird dekodiert
          line = decodeMessage(line)
          #print(line)

          #Zeile wird in ein dict konvertiert
          line= json.loads(line)

          columns = ""
          values = ""

          if line["Status"] != "OK":
                  print(line["Status"])
                  print("Skipped invalid measurements due to warm-up of the Sensor")
                  continue
          
          for type in line:
              #Diese Daten werden herausgefiltert und nicht mit in die Datenbank mit aufgenommen
              if type == "bat" or type == "DeviceID" or type == "uptime" or type == "window_event" or type == "door_event" or type == "person" or type == "window_open" or type == "Status":
                continue

              
              #Erstellt den String für die Columns
              columns += type 

              #Erstellt den String für die Values 
              #Edge-case: wenn value eine Liste ist
              if isinstance(line[type], list):
                  values += str(line[type][0])
              elif type == "timestamp":
                  values += "FROM_UNIXTIME(%s)" % (int(line[type]/1000))
              else:  
                  values += str(line[type])

              #Edge-case: Am Ende kein Komma
              if type != "cnt0_3":  
                  columns += ", "
                  values += ", "

          sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s );" % ('measurements', columns, values)
          #print(sql)
          cur.execute(sql)
          conn.commit()

In [4]:

def copy_data_from_start_date(start_date):
    current_date = datetime.date.today()
    folder = getAvailable()

    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Sortiere Ordner nach Jahr, Monat und Tag
    sorted_years = sorted(folder.keys())
    for year in sorted_years:
        sorted_months = sorted(folder[year].keys())
        for month in sorted_months:
            sorted_days = sorted(folder[year][month].keys())
            for day in sorted_days:
                sorted_files = sorted(folder[year][month][day])
                for file in sorted_files:
                    # Datumsobjekt erstellen
                    file_date = datetime.datetime.strptime(
                        year + month + day, "%Y%m%d"
                    ).date()

                    # Überprüfen, ob das Datum im gewünschten Zeitraum liegt
                    if start_date <= file_date <= current_date:
                        # Anfrage formulieren und Daten anfordern
                        print(
                            "_____________________________________________________________________________")
                        print("Year: " + year + "\n" + "Month: " + month + "\n" + "Day: " + day + "\n" + "File: " + file + "\n")  
                        connection.request("GET","/file?request="+encodeMessage(year+"/"+month+"/"+day+"/"+file))
                        contents = connection.getresponse()
                        if contents.status == 200:
                            print("Status: " + str(contents.status) + "/OK")
                        else:
                            print("Status: " + str(contents.status) + "/Canceled!")
                            break
                        sql_data(contents, cur)

    # Verbindung trennen
    connection.close()



In [5]:
# Beispielaufruf für Daten ab einem bestimmten Startdatum bis zum aktuellen Tag
start_date = datetime.date(2023, 4, 1)
copy_data_from_start_date(start_date)

OSError: [WinError 10065] Der Host war bei einem Socketvorgang nicht erreichbar